# Water Potability Class Imbalance Analysis (with Scaling)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from collections import defaultdict
import os

/tmp/ipykernel_1206/3161740821.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv("KNN_Imputed_Water_Potability_Dataset__Full_View_.csv")
X = df.drop('Potability', axis=1)
y = df['Potability']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)

In [3]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:
class_0 = X_train_scaled[y_train == 0]
class_1 = X_train_scaled[y_train == 1]
class_1_upsampled = class_1[np.random.choice(len(class_1), len(class_0), replace=True)]
X_over = np.vstack([class_0, class_1_upsampled])
y_over = pd.Series([0]*len(class_0) + [1]*len(class_1_upsampled))

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_train_scaled, y_train)

resampling_methods = {
    'Baseline': (X_train_scaled, y_train),
    'Random Oversampling': (X_over, y_over),
    'SMOTE': (X_smote, y_smote)
}

In [7]:
def evaluate_model(name, model, X_res, y_res, save_dir):
    model.fit(X_res, y_res)
    y_pred = model.predict(X_test_scaled)
    y_prob = model.predict_proba(X_test_scaled)[:, 1]

    report = classification_report(y_test, y_pred, output_dict=True)
    auc_score = roc_auc_score(y_test, y_prob)

    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(cm, display_labels=["Not Potable", "Potable"])
    disp.plot(cmap="Blues")
    plt.title(f"{name} - Confusion Matrix")
    plt.savefig(f"{save_dir}/{name}_conf_matrix.png")
    plt.close()

    fpr, tpr, _ = roc_curve(y_test, y_prob)
    plt.plot(fpr, tpr, label=f"AUC = {auc_score:.2f}")
    plt.plot([0, 1], [0, 1], linestyle="--")
    plt.title(f"{name} - ROC Curve")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend()
    plt.savefig(f"{save_dir}/{name}_roc_curve.png")
    plt.close()

    return report['1'], auc_score

In [ ]:
for model in results:
    for metric in metrics + ['AUC']:
        plt.figure()
        plt.bar(resampling_methods.keys(), results[model][metric])
        plt.ylabel(metric)
        plt.title(f"{model} - {metric}")
        plt.savefig(f"outputs/{model}_{metric}.png")
        plt.close()